# Data Validation Notebook
### Purpose: Validate the integrity of Olist marketplace data before A/B test analysis
---

In [8]:
## Setup: Import Libraries and Authenticate with BigQuery
import os
import pandas as pd
from google.cloud import bigquery, bigquery_storage
from dotenv import load_dotenv

In [9]:
# Load credentials and env variables
load_dotenv()
project_id = os.getenv("GCP_PROJECT_ID")
client = bigquery.Client(project=project_id)

## Table size checks

In [10]:
tables = [
    "orders", "order_items", "payments", "reviews",
    "customers", "sellers", "products", "geolocation"
]

for table in tables:
    query = f"""
    SELECT COUNT(*) AS row_count FROM `{project_id}.marketplace_ab_test.{table}`
    """
    result = client.query(query).to_dataframe()
    print(f"Table `{table}`: {result.row_count[0]} rows")

Table `orders`: 99441 rows
Table `order_items`: 112650 rows
Table `payments`: 103886 rows
Table `reviews`: 99224 rows
Table `customers`: 99441 rows
Table `sellers`: 3095 rows
Table `products`: 32951 rows
Table `geolocation`: 1000163 rows


In [11]:
def check_nulls(table: str, fields: list[str]) -> pd.DataFrame:
    field_checks = ", ".join([f"COUNTIF({col} IS NULL) AS null_{col}" for col in fields])
    query = f"SELECT {field_checks} FROM `{project_id}.marketplace_ab_test.{table}`"
    return client.query(query).to_dataframe()

check_nulls("orders", [
    "order_purchase_timestamp", "order_approved_at",
    "order_delivered_customer_date", "order_estimated_delivery_date"
])

,null_order_purchase_timestamp,null_order_approved_at,null_order_delivered_customer_date,null_order_estimated_delivery_date
0,0,160,2965,0


## 3. Referential Integrity Checks
### Do all payments/reviews reference valid orders?

In [12]:
# payment without matching orders
query = f"""
SELECT COUNT(*) AS unmatched_payments
FROM `{project_id}.marketplace_ab_test.payments` p
LEFT JOIN `{project_id}.marketplace_ab_test.orders` o
ON p.order_id = o.order_id
WHERE o.order_id IS NULL
"""
unmatched_payments = client.query(query).to_dataframe()
print(f"Payments Without orders: {unmatched_payments.unmatched_payments[0]}")

Payments Without orders: 0


## Reviews without matching orders

In [13]:
query = f"""
SELECT COUNT(*) AS unmatched_reviews
FROM `{project_id}.marketplace_ab_test.reviews` r
LEFT JOIN `{project_id}.marketplace_ab_test.orders` o
ON r.order_id = o.order_id
WHERE o.order_id IS NULL
"""

unmatched_reviews = client.query(query).to_dataframe()
print(f"Reviews without orders: {unmatched_reviews.unmatched_reviews[0]}")

Reviews without orders: 0


## Outlier Detection

In [14]:
query = f"""
SELECT MIN(payment_value) AS min_payment_value,
MAX(payment_value) AS max_payment_value,
AVG(payment_value) AS avg_payment_value,
STDDEV(payment_value) AS std_dev_payment_value
FROM `{project_id}.marketplace_ab_test.payments` p
"""

payment_stats = client.query(query).to_dataframe()
payment_stats

,min_payment_value,max_payment_value,avg_payment_value,std_dev_payment_value
0,0.0,13664.08,154.10038,217.494064
